<a href="https://colab.research.google.com/github/anilbhatt1/Deep_Learning_EVA4_Phase1/blob/master/Master_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Mounting the drive and copying the code from github to drive
from google.colab import drive 
drive.mount('/content/gdrive')
%cd gdrive/My Drive/project_folder
! ls

In [2]:
!git clone https://github.com/anilbhatt1/Deep_Learning_EVA4_Phase1

Cloning into 'Deep_Learning_EVA4_Phase1'...
remote: Enumerating objects: 179, done.
remote: Counting objects: 100% (179/179), done.
remote: Compressing objects: 100% (177/177), done.
remote: Total 568 (delta 113), reused 0 (delta 0), pack-reused 389
Receiving objects: 100% (568/568), 4.17 MiB | 3.18 MiB/s, done.
Resolving deltas: 100% (350/350), done.


In [3]:
%cd Deep_Learning_EVA4_Phase1/S7_CIFAR10_Modular
! ls

/content/gdrive/My Drive/project_folder/Deep_Learning_EVA4_Phase1/S7_CIFAR10_Modular
Master_Model.ipynb  models  README.md


In [0]:
from __future__ import print_function
import torch
import torchvision
from torchvision import transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from tqdm import tqdm
from models import S7_Datatransforms,S7_Model,S7_Dataloaders,S7_Train_Losses,S7_Test_losses

In [5]:
data_loader = S7_Dataloaders.DataLoader(shuffle=True,batch_size=128,seed=1)

mean = (0.49139968, 0.48215841, 0.44653091)
stdev = (0.24703223, 0.24348513, 0.26158784)
transform_instance  = S7_Datatransforms.Transforms_custom(normalize=True, mean=mean, stdev=stdev)

before_norm_list = [transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1),
               transforms.RandomRotation((-7,7),fill=(1,))]
               
test_transforms   = transform_instance.test_transforms()
train_transforms  = transform_instance.train_transforms(before_norm = None)

train_data = torchvision.datasets.CIFAR10('./data', train=True, download=True, transform=train_transforms)
test_data  = torchvision.datasets.CIFAR10('./data', train=False, download=True, transform=test_transforms)

classes = ('plane', 'car', 'bird', 'cat','deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [0]:
trainloader = data_loader.load(train_data)
testloader  = data_loader.load(test_data)

In [7]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)
model = S7_Model.Net()
model.to(device)
summary(model, input_size=(3, 32, 32))

cuda
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 32, 32]             864
              ReLU-2           [-1, 32, 32, 32]               0
       BatchNorm2d-3           [-1, 32, 32, 32]              64
           Dropout-4           [-1, 32, 32, 32]               0
            Conv2d-5           [-1, 64, 32, 32]          18,432
              ReLU-6           [-1, 64, 32, 32]               0
       BatchNorm2d-7           [-1, 64, 32, 32]             128
           Dropout-8           [-1, 64, 32, 32]               0
         MaxPool2d-9           [-1, 64, 16, 16]               0
           Conv2d-10           [-1, 32, 16, 16]           2,048
           Conv2d-11           [-1, 64, 16, 16]          18,432
             ReLU-12           [-1, 64, 16, 16]               0
      BatchNorm2d-13           [-1, 64, 16, 16]             128
          Dropout-14           [-1

In [0]:
train_loss = S7_Train_Losses.Train_loss()
test_loss  = S7_Test_losses.Test_loss()

# Train and Test the model 

In [9]:
EPOCH     = 20
L2_factor = 0.0005
L1_factor = 0.001
OPTIMIZER = optim.SGD(model.parameters(), lr=0.025, momentum=0.9,weight_decay=L2_factor)
train_loss_all = []
train_acc_all  = []
test_loss_all  = []
test_acc_all   = []

for epoch in range(EPOCH):

    print("EPOCH:", epoch)
    train_losses, train_acc = train_loss.train_loss_calc(model,device,train_loader=trainloader,optimizer=OPTIMIZER,epoch=EPOCH,factor=L1_factor)
    test_losses, test_acc   = test_loss.test_loss_calc(model,device,test_loader=testloader)

    train_loss_all.extend(train_losses)
    train_acc_all.extend(train_acc)
    test_loss_all.extend(test_losses)
    test_acc_all.extend(test_acc)

  0%|          | 0/391 [00:00<?, ?it/s]

EPOCH: 0


Train Loss = 1.2024728059768677 Batch Id = 390 Train Accuracy = 46.66: 100%|██████████| 391/391 [00:42<00:00,  9.14it/s]
  0%|          | 0/391 [00:00<?, ?it/s]


Test set: Average loss: 154.9283, Test Accuracy: 70.29

EPOCH: 1


Train Loss = 0.866962730884552 Batch Id = 390 Train Accuracy = 62.36: 100%|██████████| 391/391 [00:42<00:00,  9.19it/s]
  0%|          | 0/391 [00:00<?, ?it/s]

...


Test set: Average loss: 76.2167, Test Accuracy: 101.68

EPOCH: 18


Train Loss = 0.533661425113678 Batch Id = 390 Train Accuracy = 84.02: 100%|██████████| 391/391 [00:42<00:00,  9.14it/s]
  0%|          | 0/391 [00:00<?, ?it/s]


Test set: Average loss: 76.9379, Test Accuracy: 101.14

EPOCH: 19


Train Loss = 0.3992951512336731 Batch Id = 390 Train Accuracy = 84.18: 100%|██████████| 391/391 [00:42<00:00,  9.22it/s]



Test set: Average loss: 79.7730, Test Accuracy: 100.05

